<a href="https://colab.research.google.com/github/juooo1117/practice_AI_Learning/blob/main/DeepLearing_SOM_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Going from Unsupervised to Supervised Deep Learning

In [1]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Creating the matrix of features

In [3]:
# importing the dataset
dataset = pd.read_csv('/content/MyDrive/MyDrive/Credit_Card_Applications.csv')
som_frauds = pd.read_csv('/content/MyDrive/MyDrive/som_frauds.csv')

In [4]:
# Class column
## 0: application is not approved, 1: application is approved
dataset

,CustomerID,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,Class
0,15776156,1,22.08,11.460,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,15739548,0,22.67,7.000,2,8,4,0.165,0,0,0,0,2,160,1,0
2,15662854,0,29.58,1.750,1,4,4,1.250,0,0,0,1,2,280,1,0
3,15687688,0,21.67,11.500,1,5,3,0.000,1,1,11,1,2,0,1,1
4,15715750,1,20.17,8.170,2,6,4,1.960,1,1,14,0,2,60,159,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,15808223,1,31.57,10.500,2,14,4,6.500,1,0,0,0,2,0,1,1
686,15769980,1,20.67,0.415,2,8,4,0.125,0,0,0,0,2,0,45,0
687,15675450,0,18.83,9.540,2,6,4,0.085,1,0,0,0,2,100,1,1
688,15776494,0,27.42,14.500,2,14,8,3.085,1,1,1,0,2,120,12,1


In [6]:
# feature matrix: 은행의 모든 고객 정보를 담고 있음 (each row has the information of one customer)
## fraud=1 (there is the fraud) / fraud=0 (there is no fraud)
customers = dataset.iloc[:,1:].values  # except the first column
customers

array([[1.000e+00, 2.208e+01, 1.146e+01, ..., 1.000e+02, 1.213e+03,
        0.000e+00],
       [0.000e+00, 2.267e+01, 7.000e+00, ..., 1.600e+02, 1.000e+00,
        0.000e+00],
       [0.000e+00, 2.958e+01, 1.750e+00, ..., 2.800e+02, 1.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 1.883e+01, 9.540e+00, ..., 1.000e+02, 1.000e+00,
        1.000e+00],
       [0.000e+00, 2.742e+01, 1.450e+01, ..., 1.200e+02, 1.200e+01,
        1.000e+00],
       [1.000e+00, 4.100e+01, 4.000e-02, ..., 5.600e+02, 1.000e+00,
        1.000e+00]])

#Creating the dependent variable

In [18]:
# the result of minisom
som_frauds

array([[1.5681755e+07, 1.0000000e+00, 1.9000000e+01, 1.7500000e+00,
        1.0000000e+00, 8.0000000e+00, 4.0000000e+00, 2.3350000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        2.0000000e+00, 1.1200000e+02, 7.0000000e+00],
       [1.5690153e+07, 1.0000000e+00, 3.4000000e+01, 5.5000000e+00,
        1.0000000e+00, 8.0000000e+00, 4.0000000e+00, 1.5000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        2.0000000e+00, 6.0000000e+01, 1.0000000e+00],
       [1.5795690e+07, 1.0000000e+00, 3.1580000e+01, 7.5000000e-01,
        1.0000000e+00, 6.0000000e+00, 4.0000000e+00, 3.5000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        2.0000000e+00, 3.2000000e+02, 1.0000000e+00],
       [1.5711635e+07, 1.0000000e+00, 3.6170000e+01, 4.2000000e-01,
        1.0000000e+00, 9.0000000e+00, 4.0000000e+00, 2.9000000e-01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        2.0000000e+00,

In [19]:
# initializing the vector
## len(dataset) : number of observation
is_fraud = np.zeros(len(dataset))
for i in range(len(dataset)):
  if dataset.iloc[i,0] in som_frauds:   # iloc[i,0] : id number of i번째 customer
    is_fraud[i] = 1                     # fruad list(; som_frauds)에 있다면 0 -> 1로 변경

# Build second Deeplearing model to supervise previous Deeplearning one

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
customers = sc.fit_transform(customers)

In [21]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

classifier.add(Dense(units = 2,
                     kernel_initializer = 'uniform',
                     activation = 'relu',
                     input_dim = 15))

classifier.add(Dense(units = 1,
                     kernel_initializer = 'uniform',
                     activation = 'sigmoid'))

classifier.compile(optimizer = 'adam',
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'])

classifier.fit(customers, is_fraud, batch_size = 1, epochs = 2)

Epoch 1/2
690/690 [==============================] - 2s 2ms/step - loss: 0.4487 - accuracy: 0.9696
Epoch 2/2
690/690 [==============================] - 1s 2ms/step - loss: 0.1605 - accuracy: 0.9696


In [27]:
# Predicting the probabilities of frauds
y_pred = classifier.predict(customers)
y_pred

22/22 [==============================] - 0s 2ms/step


array([[0.06381314],
       [0.05864867],
       [0.07171382],
       [0.09627633],
       [0.03708248],
       [0.0569631 ],
       [0.03903377],
       [0.0266882 ],
       [0.05371654],
       [0.06647723],
       [0.03522813],
       [0.03651359],
       [0.17623636],
       [0.02160914],
       [0.06441408],
       [0.04231622],
       [0.05332446],
       [0.06377216],
       [0.04172682],
       [0.05195819],
       [0.05707736],
       [0.10962415],
       [0.03839537],
       [0.02560015],
       [0.04075059],
       [0.0289731 ],
       [0.04067149],
       [0.12068541],
       [0.05440001],
       [0.04947918],
       [0.0501259 ],
       [0.14670284],
       [0.08132745],
       [0.03065141],
       [0.15607484],
       [0.09949439],
       [0.05081121],
       [0.02549412],
       [0.03418394],
       [0.04682096],
       [0.03036887],
       [0.06918242],
       [0.0497308 ],
       [0.08698589],
       [0.14374448],
       [0.05212778],
       [0.06169439],
       [0.043

In [28]:
# adding the customer ID as the second column
y_pred = np.concatenate((dataset.iloc[:, 0:1].values, y_pred), axis=1)

In [30]:
# customer ID, predicted probabilities
## probabilities need to sort (low probability -> high probability)

y_pred = y_pred[y_pred[:, 1].argsort()]
result = pd.DataFrame(y_pred)
result

,0,1
0,15799785.0,0.004940
1,15621423.0,0.006311
2,15654859.0,0.006902
3,15585855.0,0.006957
4,15723827.0,0.009000
...,...,...
685,15782159.0,0.204002
686,15799859.0,0.210399
687,15736399.0,0.214650
688,15670029.0,0.241516
